In [ ]:
!pip install  fasttext numpy

In [ ]:
!pip install fasttext numpy==1.26.4 --ignore-installed

  Using cached fasttext-0.9.3-cp311-cp311-linux_x86_64.whl
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached setuptools-80.3.1-py3-none-any.whl.metadata (6.5 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
Using cached setuptools-80.3.1-py3-none-any.whl (1.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
numpy.__version__

NameError: name 'numpy' is not defined

In [ ]:
import fasttext
import numpy as np

In [ ]:
import pandas as pd
from google.colab import files

In [ ]:
def prepare_data(file_path):
    df = pd.read_csv(file_path)
    df["text"] = "__label__" + df["LFFUNC"] + "_" + df["LFVAL"] + " " + df["LFARG"]
    df["text"].to_csv("fasttext_train.txt", index=False, header=False)

    return df

In [ ]:
def train_model():
    model = fasttext.train_supervised(
        input="fasttext_train.txt",
        lr=0.1,
        epoch=100,
        wordNgrams=2,
        dim=100,
        verbose=2
    )
    return model

In [ ]:
def predict_lf(model, function, word, top_k=5):
    labels, probs = model.predict(word, k=1000)

    results = []
    for label, prob in zip(labels, probs):
        if f"__label__{function}_" in label:
            lf_val = label.replace(f"__label__{function}_", "")
            results.append((lf_val, prob))

    return sorted(results, key=lambda x: -x[1])[:top_k]

In [ ]:
if __name__ == "__main__":
    uploaded = files.upload()
    file_name = next(iter(uploaded))

    df = prepare_data(file_name)
    model = train_model()

    test_cases_old = [
        ('MAGN', 'довод'),
        ('OPER1', 'домино'),
        ('OPER2', 'арест'),
        ('INCEPOPER1', 'азарт'),
        ('FUNC0', 'дорога'),
        ('INCEPFUNC0', 'день'),
        ('CAUSFUNC0', 'встреча'),
        ('REAL1', 'газета'),
        ('REAL1-M', 'долг')
    ]

    test_cases_new = [
        ('LOC', 'дом'),
        ('OPER1', 'оценка'),
        ('MAGN', 'друг'),
        ('ADV2-UN', 'причина'),
        ('ADV1-UN', 'надежда'),
        ('CAUSFUNC0', 'заседание'),
        ('FUNC0', 'дело'),
        ('INCEPOPER1', 'право'),
        ('OPER2', 'внимание')
    ]

    for func, arg in test_cases_old:
        print(f"\nТоп-5 вариантов для {func}({arg}):")
        predictions = predict_lf(model, func, arg)

        if not predictions:
            print("Нет предсказаний для этой комбинации")
        else:
            for i, (val, prob) in enumerate(predictions, 1):
                print(f"{i}. {val} (вероятность: {prob:.4f})")

Saving lf_data_words.csv to lf_data_words (4).csv


TypeError: Cannot convert numpy.ndarray to numpy.ndarray